In [291]:
import numpy as np

import pandas as pd
import uuid

import matplotlib.pyplot as plt
import matplotlib.animation as animation
from IPython.display import HTML

In [677]:
class ptype:
    """basic particle type, with name and physical properties
    it also has a list of unimolecular reactions A -> products
    """
    def __init__(self,diffusion, name):
        self.diffusion=diffusion
        self.name=name
        self.unimolecular_reactions = []
        self.unimolecular_probs = []
        
    def add_reaction(self,reaction):
        """adds a unimolecular_reaction to the list for this particle type
        and updates the probabilities function"""
        (self.unimolecular_reactions).append(reaction)
        rates = np.array([rea.rate for rea in self.unimolecular_reactions])
        self.unimolecular_probs = rates
        
    def choose_reaction(self,dt):
        """chooses one reaction (or none) for a given time step
        that reaction can then be effected on a particle with effect_reaction
        """
        if len(self.unimolecular_probs)==0:
            return None
        else:
            rates = self.unimolecular_probs
            probs = (rates/rates.sum())*(1 - np.exp( - dt* rates.sum()))
            #print("Cumulative probs: ",np.cumsum(probs))
            alpha=np.random.uniform(size=1)
            conf = np.cumsum(probs)>alpha
            if any(conf):
                ii = np.argmax(conf)
                print("Chosen rection %s" % self.unimolecular_reactions[ii].name)
                return self.unimolecular_reactions[ii]
            else:
                return None
        
class particle:
    """particle instance, it has an id, position, age, types, etc."""
    def __init__(self, ptype, pos, age = 0, id = None ):
        if id==None:
            self.id = uuid.uuid4()
        else:
            self.id = id
        self.ptype = ptype
        self.age = age
        self.pos = pos
        
    def step(self,dt):
        pos1 = self.pos + np.sqrt(2*self.ptype.diffusion*dt)*np.random.normal(0, 1, 2)
        #print("type %s, age %d"% (self.ptype.name, self.age))
        pnew = particle(self.ptype, pos1, age = self.age + 1, id = self.id)
        return pnew

        
class traj:
    """a traj is a list of lists of particles"""
    
    def __init__(self, particles):
        self.ptraj = []
        self.nframes = 0
        (self.ptraj).append(particles)
        
    def step(self,dt):
        newvals=[]
        for pa in (self.ptraj)[self.nframes]:
            pa1 = pa.step(dt)
            newvals.append(pa1)
        (self.ptraj).append(newvals)
        self.nframes = (self.nframes)+1

    def react_unimolecular(self, dt):
        """loops over particles and reacts them with their unimolecular reactions
         unimolecular reactions are associated to the incoming pair
        """
        newvals=[]
        for pa in (self.ptraj)[self.nframes]:
            react = pa.ptype.choose_reaction(dt)
            if(not react==None):
                pa1 = react.effect_reaction(pa)
                if not pa1==None:
                    for pa11 in pa1:
                        newvals.append(pa11)
            else:
                newvals.append(pa)
        
        (self.ptraj)[self.nframes] = newvals
        
    def react_bimolecular(self,dt):
        """ bimolecular reactions
        
        """

In [293]:
def plt_traj(traj):
    """plots a trajectory"""
    ptraj = (traj.ptraj)
    fig = plt.figure()
    ax = plt.axes(xlim=(-4, 4), ylim=(-4, 4))
    line, = ax.plot([], [], marker='o', lw=2)
        
    def init():
        line.set_data([], [])
        return line,
        
    def animate(i):
        x = [p.pos[0] for p in ptraj[i]]
        y = [p.pos[1] for p in ptraj[i]]
        line.set_data(x, y)
        return line, 

    anim = animation.FuncAnimation(fig, animate, init_func=init,
                               frames=traj.nframes, interval=20, blit=True)
    return anim

 

In [660]:
def reshape(traj):
    """converts a frame-based trajectory into a id-based data-frame
    for lifetimes and concentrations
    
    """
    pp=[]
    for i,frame in enumerate(traj.ptraj):
        for part in frame:
            pp.append([part.id,part.age,part.pos[0],part.pos[1],i,part.ptype.name])
    gg = pd.DataFrame(pp,columns=['ID','age','x','y','frame','type_name'])
    return gg


In [633]:
class reaction:
    def __init__(self,name,rate):
        self.rate = rate
        self.name = name
        #print(self)

### ============== UNI_MOLECULAR REACTIONS ===============================
        
class decay_reaction(reaction):
    """ unimolecular decay with given rate"""
    def __init__(self, name, rate):
        super(decay_reaction,self).__init__(name,rate)
        
    def effect_reaction(self,particle_in):
        return None
    
class convert_reaction(reaction):
    """ unimolecular conversion with given rate to given type """
    def __init__(self,  name, rate , outtype):
        self.outtype=outtype
        super(convert_reaction,self).__init__( name, rate)
        
    def effect_reaction(self,particle_in):
        ## return a singleton list for consistence with other reactions
        return [particle(self.outtype, particle_in.pos )]

class fission_reaction(reaction):
    """ unimolecular fission with given rate to given type """
    def __init__(self,  name, rate, outtype1, outtype2):
        self.outtype1=outtype1
        self.outtype2=outtype2
        super(fission_reaction,self).__init__( name, rate)
        
    def effect_reaction(self,particle_in):
        pos = particle_in.pos
        return [particle(self.outtype1, pos) ,particle(self.outtype2, pos ) ]  

In [ ]:
p1 = ptype(1.0e-2,"type 1")
r1 = decay_reaction("dec1",1e-5)
r2 = fission_reaction('fission1',1e-4, p1,p2)
p1.add_reaction(fission_reaction("reac1",1e3, p1,p2))
p1.add_reaction(fission_reaction("reac3",1e3, p1,p2))
p1.add_reaction(fission_reaction("reac2",1e3, p1,p2))

r=p1.choose_reaction(0.0002)
if not r==None:
    print(r.name)
    
news = r.effect_reaction(init_pos[0])
news

alpha=np.random.uniform(size=1)
alpha

ppppp =np.cumsum(rrr/rrr.sum()*(1-np.exp(-0.00002*rrr.sum())))
ppppp


In [724]:
class encunter_complex:
    def __init__(self, name):
        self.name=name
        
class bim_encunter_complex(encunter_complex):
    def __init__(self, name, typ1,typ2, dist):
        self.encunter_complex=encunter_complex
        self.type1 = typ1
        self.type2 = typ2
        self.dist= dist
        super(bim_encunter_complex,self).__init__(name)

    def get_ec(self,particles_):
        p1 = [i for i,p in enumerate(particles_) if p.ptype==self.type1]
        p2 = [i for i,p in enumerate(particles_) if p.ptype==self.type2]
        ec=[]
        #print(p1)
        #print(p2)
        for q1 in p1:
            for q2 in p2:
                if not q1 == q2:
                    vec1= particles_[q1].pos - particles_[q2].pos
                    #print(vec1)
                    dist1= np.sqrt((vec1 * vec1).sum())
                    if dist1 < self.dist:
                        ec.append([q1,q2])
        return ec

In [739]:
b1=bim_encunter_complex("bim1",p1,p3,.9)

In [742]:
b1.get_ec(t.ptraj[100])

[[1, 0]]

In [ ]:
class fusion_reaction(reaction):
    """ fusion reaction"""
    def __init__(self, name, rate, encounter_complex, outtpye):
        self.encunter_complex=encunter_complex
        self.outtype = outtpye
        super(fusion_reaction,self).__init__(name,rate)
        
    def effect_reaction(self,encunter_complex_in):
        return None

### Run 

In [668]:

## create particle types and their unimolecular reactions
p1 = ptype(1.0e-2,"type 1")
p2 = ptype(5.0e-1,"type 2")
p3 = ptype(1.0e-2,"type 1b")
p2.add_reaction( decay_reaction("dec1",1.1) )
p1.add_reaction( convert_reaction("con1",.1, p3) )




init_pos = [
    particle(p1, np.array([0,i]),age=0) for i in range(10)] + [
    particle(p2, np.array([i,0]),age=0) for i in range(10)] 
t = traj(init_pos)

In [669]:
for i in range(1000):
    t.step(.1)
    t.react_unimolecular(.1)

Chosen rection dec1
Chosen rection dec1
Chosen rection dec1
Chosen rection dec1
Chosen rection con1
Chosen rection con1
Chosen rection dec1
Chosen rection dec1
Chosen rection dec1
Chosen rection dec1
Chosen rection dec1
Chosen rection dec1
Chosen rection con1
Chosen rection con1
Chosen rection con1
Chosen rection con1
Chosen rection con1
Chosen rection con1
Chosen rection con1
Chosen rection con1


In [678]:
t.ptraj[10]

In [241]:
aa = plt_traj(t)
#HTML(aa.to_html5_video())

In [670]:
tr=reshape(t)
pp=tr.groupby(by='ID')
pp.agg({'age':'max', 'type_name':'unique'})

,type_name,age
ID,,
001a548c-b86f-4b13-b57d-86258e17427a,[type 1],46
06cc09fa-04ac-4b37-a0a0-567034a2d3a3,[type 1b],920
263fc5ab-4704-448c-873c-563bdf2a76ff,[type 1],66
37cb835d-8b74-4de8-a9ef-0358ca5764fd,[type 2],13
3e98d294-3d83-4557-a47f-22dd12fcfe70,[type 1b],994
440705f6-6fa8-4aee-8f33-6e1916c6cf30,[type 1b],933
4409f274-c2d9-4fb3-8493-ac86793aa727,[type 1b],950
524789ae-2860-447c-a8fd-f429f201b994,[type 2],8
55ca40bb-90cb-4fb6-a492-817bc75da443,[type 1b],953


In [645]:
lifes = pp.agg({'age':'max', 'type_name':'unique'})
yy = lifes.groupby('type_name')
#for ky, val in yy:
#    print(val)
yy

In [654]:
lifes

,type_name,age
ID,,
08e9616f-7a76-4356-8b68-3d8b4805c46f,[type 1],72
12acc033-cd43-4968-8fd7-9d66981fd89f,[type 1],14
1c5aa45f-1d71-4ef0-abf6-a5309fb887e4,[type 1],108
2c2922f0-436d-44c2-b7f0-efffe63f069a,[type 2],2
300b6afb-8ed5-45ed-8b86-bcc55e9ac3c0,[type 2],2
353c3abc-a155-4cc5-b465-8b1986f3b8a0,[type 2],7
4b5af5c9-cc49-4501-82f3-806105d05498,[type 1],72
5b6a478a-1f1c-4698-9e3d-b41c87b9f510,[type 1b],938
65918629-bf1c-48a5-bf63-097794abcc61,[type 2],2


In [676]:
pp2=tr.groupby(by=['frame','type_name'])
qq2=pp2.agg({'type_name':'count'})

In [675]:
qq2

type_name
frame type_name           
0     type 1            10
      type 2            10
1     type 1            10
      type 2             7
2     type 1            10
      type 2             6
3     type 1            10
      type 2             6
4     type 1            10
      type 2             6
5     type 1            10
      type 2             6
6     type 1             9
      type 1b            1
      type 2             6
7     type 1             9
      type 1b            1
      type 2             6
8     type 1             8
      type 1b            2
      type 2             5
9     type 1             8
      type 1b            2
      type 2             4
10    type 1             8
      type 1b            2
      type 2             4
11    type 1             8
      type 1b            2
      type 2             4
...                    ...
971   type 1b           10
972   type 1b           10
973   type 1b           10
974   type 1b           10
975   type 1b           10
976   type 1b           10
977   type 1b           10
978   type 1b           10
979   type 1b           10
980   type 1b           10
981   type 1b           10
982   type 1b           10
983   type 1b           10
984   type 1b           10
985   type 1b           10
986   type 1b           10
987   type 1b           10
988   type 1b           10
989   type 1b           10
990   type 1b           10
991   type 1b           10
992   type 1b           10
993   type 1b           10
994   type 1b           10
995   type 1b           10
996   type 1b           10
997   type 1b           10
998   type 1b           10
999   type 1b           10
1000  type 1b           10

[1210 rows x 1 columns]